In [10]:
import os
import csv
import ast
import pandas as pd
import numpy as np
import logging
import json
import re
from bs4 import BeautifulSoup
from typing import Any
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
import warnings
import sys
import html

# ======== PATHS ========
BASE_PATH = r"C:\Users\zalma"
A_PATH = os.path.join(BASE_PATH, "A")
B_PATH = os.path.join(BASE_PATH, "B")
C_PATH = os.path.join(BASE_PATH, "C")
OUTPUT_PATH = os.path.join(BASE_PATH, "merge")

# ======== LOGGING CONFIGURATION ========
os.makedirs(OUTPUT_PATH, exist_ok=True)
LOG_FILE = os.path.join(OUTPUT_PATH, "merge_log.txt")

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] %(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.FileHandler(LOG_FILE, encoding="utf-8"), logging.StreamHandler()],
)

# ======== HELPER FUNCTIONS ========
def load_csv_safely(path: str, **kwargs: Any) -> pd.DataFrame:
    """
    Betölt egy CSV fájlt, hiba esetén üres DataFrame-et ad vissza.
    """
    try:
        df = pd.read_csv(path, **kwargs)
        logging.info(f"Loaded: {os.path.basename(path)} ({len(df)} rows)")
        return df
    except Exception as e:
        logging.error(f"Error loading {path}: {e}")
        return pd.DataFrame()


def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardizálja a DataFrame oszlopneveit: levágja a szóközöket, kisbetűssé alakítja,
    és helyettesíti a szóközöket és kötőjeleket alulvonással.
    """
    df.columns = (
        df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")
    )
    return df

def normalize_tags_column(val):
    """
    Egységesíti a 'tags' oszlop értékeit:
    - Ha dict-string: {'Action': 5472, 'FPS': 4897} → [{"tag_name": "Action", "weight": 5472}, ...]
    - Ha már list-of-dict, érintetlenül hagyja.
    """
    if isinstance(val, str):
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, dict):
                return [{"tag_name": k, "weight": v} for k, v in parsed.items()]
        except Exception:
            pass
    return val


def fill_missing_from_source(D: pd.DataFrame, src: pd.DataFrame) -> pd.DataFrame:
    """
    Kitölti a hiányzó értékeket a D DataFrame-ben egy forrás (src) adatai alapján.
    Az appid oszlop alapján merge-öl, a közös oszlopokat balról tölti.
    """
    src = src.copy()
    src["appid"] = src["appid"].astype(str)

    common_cols = [col for col in src.columns if col in D.columns]

    merged = D.merge(
        src[common_cols],
        on="appid",
        how="left",
        suffixes=("", "_src")
    )

    for col in common_cols:
        if col != "appid":
            merged[col] = merged[col].combine_first(merged[f"{col}_src"])
            merged.drop(columns=[f"{col}_src"], inplace=True)

    return merged

def clean_html_entities(text: str) -> str:
    """Eltávolítja a HTML tageket és dekódolja az entitásokat (pl. &reg; → ®)."""
    if pd.isna(text):
        return ""
    soup = BeautifulSoup(str(text), "html.parser")
    cleaned = soup.get_text(" ", strip=True)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned
    
def clean_language_field_merged(val):
    """
    Normalizálja a supported_languages / full_audio_languages mezőt:
    - Tisztítja a HTML-, BBCode- és Steam-maradványokat
    - Szétbontja az összefűzött nyelveket (, ; szóköz)
    - Egyesíti az egyszerű HTML-tisztítást és a nyelvnév normalizálást
    """
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return []

    if isinstance(val, (list, tuple, np.ndarray)):
        raw_items = [str(v).strip() for v in val if str(v).strip()]
    else:
        val_str = str(val).strip()
        if not val_str:
            return []
        try:
            parsed = ast.literal_eval(val_str)
            if isinstance(parsed, list):
                raw_items = [str(v).strip() for v in parsed if v]
            else:
                raw_items = [val_str]
        except Exception:
            raw_items = [val_str]

    cleaned = []
    for x in raw_items:
        x = html.unescape(x)
        x = re.sub(r"<[^>]+>", " ", x)
        x = re.sub(r"&lt;/?\w+&gt;", " ", x)
        x = re.sub(r"&nbsp;", " ", x)
        x = re.sub(r"#lang[_\-]?", "", x, flags=re.IGNORECASE)
        x = re.sub(r"\[/?[^\]]+\]", "", x)
        x = re.sub(r"&[a-z]+;", " ", x)
        x = re.sub(r"\s+", " ", x).strip()
        if x:
            cleaned.extend(re.split(r"[,;/]", x))

    cleaned = [c.strip() for c in cleaned if c.strip()]
    seen = set()
    result = []
    for c in cleaned:
        key = c.lower()
        if key not in seen:
            seen.add(key)
            result.append(c)

    return result

def final_clean_language_list(langs: list[str]) -> list[str]:
    """
    Utólagos tisztítás egy játék nyelvlistáján:
    - HTML/BBCode/technikai morzsák eltávolítása
    - Tipikus kettős elemek összevonása (Simplified/Traditional Chinese, Spanish – Spain/LatAm, Portuguese – Brazil/Portugal)
    - Duplikátumok kiszűrése
    - LAZÁBB validáció (engedjük a kötőjelet, pontot, vesszőt, számot is; 2+ karakter)
    - Csak a nyilvánvaló szemét (lt, gt, br, strong, audio, support, text, /br) kiszűrése
    """
    import re, html

    if not langs or not isinstance(langs, list):
        return []

    cleaned = []
    for name in langs:
        if not name or not isinstance(name, str):
            continue
        n = html.unescape(name)
        n = re.sub(r"<[^>]*>", " ", n)
        n = re.sub(r"&lt;?/?\w+&gt;?", " ", n)
        n = re.sub(r"&[a-z]+;", " ", n)
        n = re.sub(r"[\(\)]", " ", n)
        n = re.sub(r"#lang[_\-]?", " ", n, flags=re.IGNORECASE)
        n = re.sub(r"\[/?[^\]]+\]", " ", n)

        n = re.sub(r"\b(?:lt|gt|strong|br|/br)\b", " ", n, flags=re.IGNORECASE)

        n = re.sub(r"\s+", " ", n).strip()
        if not n:
            continue

        if n.isupper():
            n = n.capitalize()

        cleaned.append(n)

    joined = []
    skip = False
    for i, word in enumerate(cleaned):
        if skip:
            skip = False
            continue
        w = word.strip()
        nxt = cleaned[i + 1].strip().lower() if i < len(cleaned) - 1 else ""

        def join2(a, b):
            return f"{a} {b}"

        low = w.lower()
        if i < len(cleaned) - 1:
            if low == "simplified" and nxt == "chinese":
                joined.append("Simplified Chinese"); skip = True; continue
            if low == "traditional" and nxt == "chinese":
                joined.append("Traditional Chinese"); skip = True; continue
            if low == "spanish" and "spain" in nxt:
                joined.append("Spanish - Spain"); skip = True; continue
            if low == "spanish" and "latin" in nxt:
                joined.append("Spanish - Latin America"); skip = True; continue
            if low == "portuguese" and "brazil" in nxt:
                joined.append("Portuguese - Brazil"); skip = True; continue
            if low == "portuguese" and "portugal" in nxt:
                joined.append("Portuguese - Portugal"); skip = True; continue

        joined.append(w)

    standardized = []
    for lang in joined:
        s = lang
        s = s.replace("Spanish Spain", "Spanish - Spain")
        s = s.replace("Spanish Latin America", "Spanish - Latin America")
        s = s.replace("Portuguese Brazil", "Portuguese - Brazil")
        s = s.replace("Portuguese Portugal", "Portuguese - Portugal")
        s = re.sub(r"\s+", " ", s).strip()
        standardized.append(s)

    valid_lang_pattern = re.compile(r"^[A-Za-zÀ-ÿ0-9' .,\-]{2,}$")

    hard_exclude = re.compile(r"\b(?:lt|gt|br|strong|audio|support|text|/br)\b", flags=re.IGNORECASE)

    corrections_map = {
        r"\bhe ew\b": "Hebrew",
        r"\bma ese\b": "Maltese",
        r"\bazil\b": "Brazil",
        r"\bfran(?:c|ç)ais\b": "Français",
    }

    fixed = []
    for lang in standardized:
        if hard_exclude.search(lang):
            continue

        s = lang
        for wrong_re, right in corrections_map.items():
            s = re.sub(wrong_re, right, s, flags=re.IGNORECASE)

        s = re.sub(r"\s+", " ", s).strip()

        if valid_lang_pattern.match(s):
            fixed.append(s)

    seen = set()
    out = []
    for x in fixed:
        k = x.lower()
        if k not in seen:
            seen.add(k)
            out.append(x)
    return out


# ======== SOURCE LOADING FUNCTIONS ========
def load_source_a(a_path: str) -> pd.DataFrame:
    """
    Betölti az A forrást (Steam CSV fájlok), megtisztítja az oszlopneveket,
    és merge-eli a különböző fájlokat egy DataFrame-be.
    """
    steam = load_csv_safely(os.path.join(a_path, "steam.csv"))
    description = load_csv_safely(os.path.join(a_path, "steam_description_data_cleaned.csv"))
    media = load_csv_safely(os.path.join(a_path, "steam_media_data.csv"))
    support = load_csv_safely(os.path.join(a_path, "steam_support_info.csv"))
    tags = load_csv_safely(os.path.join(a_path, "steamspy_tag_data.csv"))
    reqs = load_csv_safely(os.path.join(a_path, "steam_requirements_data.csv"))

    for df in [steam, description, media, support, tags, reqs]:
        if not df.empty:
            df = clean_columns(df)
            possible_ids = [c for c in df.columns if "appid" in c.lower()]
            if possible_ids:
                df.rename(columns={possible_ids[0]: "appid"}, inplace=True)
    if not tags.empty:
        tag_cols = [c for c in tags.columns if c != "appid" and tags[c].dtype in [int, float]]
        if tag_cols:
            melted = tags.melt(
                id_vars=["appid"],
                value_vars=tag_cols,
                var_name="tag_name",
                value_name="weight"
            )
            melted = melted[melted["weight"] > 0]
            tags_dict = (
                melted.groupby("appid")
                .apply(lambda x: {t: int(w) for t, w in zip(x["tag_name"], x["weight"])})
                .to_dict()
            )
            tags = pd.DataFrame({"appid": list(tags_dict.keys()), "tags": list(tags_dict.values())})
            logging.info(f"SteamSpy tags converted → {len(tags)} appid with tag data")
        else:
            logging.warning("No numeric tag columns found in steamspy_tag_data.csv")

    merged = (
        steam.merge(description, on="appid", how="left")
        .merge(media, on="appid", how="left")
        .merge(support, on="appid", how="left")
        .merge(tags, on="appid", how="left")
        .merge(reqs, on="appid", how="left")
    )
    logging.info(f"A source merged: {len(merged)} rows")
    return merged


def load_source_b(base_path: str) -> pd.DataFrame:
    """
    Betölti a B forrást JSON fájlból, előkészíti Pandas DataFrame-re,
    és beállítja a numerikus és logikai oszlopok típusait.
    """
    file_path = os.path.join(base_path, "games.json")

    if not os.path.exists(file_path):
        logging.error(f"File not found: {file_path}")
        return pd.DataFrame()

    with open(file_path, "r", encoding="utf-8") as f:
        dataset = json.load(f)

    records = []
    for appID, game in dataset.items():
        fields = [
            "name",
            "release_date",
            "estimated_owners",
            "price",
            "required_age",
            "dlc_count",
            "detailed_description",
            "short_description",
            "about_the_game",
            "reviews",
            "header_image",
            "website",
            "support_url",
            "support_email",
            "windows",
            "mac",
            "linux",
            "metacritic_score",
            "metacritic_url",
            "user_score",
            "positive",
            "negative",
            "score_rank",
            "achievements",
            "recommendations",
            "notes",
            "average_playtime_forever",
            "average_playtime_2weeks",
            "median_playtime_forever",
            "median_playtime_2weeks",
            "peak_ccu",
        ]

        record = {key: game.get(key) for key in fields}
        record["appid"] = str(appID)

        list_fields = [
            "packages", "developers", "publishers", "categories", "genres",
            "supported_languages", "full_audio_languages", "screenshots", "movies"
        ]
        record.update({f: game.get(f, []) for f in list_fields})
        
        tags = game.get("tags", {})
        record["tags"] = tags if isinstance(tags, dict) else {}


        records.append(record)

    df_b = pd.DataFrame(records)

    if "release_date" in df_b.columns:
        df_b["release_date"] = pd.to_datetime(df_b["release_date"], errors="coerce")
        df_b["release_date"] = df_b["release_date"].dt.strftime("%Y-%m-%d")
        df_b["release_date"] = df_b["release_date"].replace("NaT", None)

    df_b_exploded = df_b.explode("packages").dropna(subset=["packages"])

    numeric_cols = [
        "metacritic_score",
        "user_score",
        "positive",
        "negative",
        "achievements",
        "recommendations",
        "price",
        "required_age",
        "dlc_count",
        "average_playtime_forever",
        "average_playtime_2weeks",
        "median_playtime_forever",
        "median_playtime_2weeks",
        "peak_ccu",
    ]
    
    packages_df = pd.json_normalize(df_b.explode("packages")["packages"])
    
    for col in numeric_cols:
        if col in df_b.columns:
            df_b[col] = pd.to_numeric(df_b[col], errors="coerce")

    bool_cols = ["windows", "mac", "linux"]
    for col in bool_cols:
        if col in df_b.columns:
            df_b[col] = df_b[col].astype(bool)

    logging.info(f"B source loaded from JSON: {len(df_b)} rows")
    return df_b


def load_source_c(c_path: str) -> pd.DataFrame:
    """
    Betölti a C forrást több CSV fájlból, megtisztítja az oszlopneveket,
    és egyesíti az adatokat egy DataFrame-be.
    """
    c_files = [
        "games_march2025_cleaned.csv",
        "games_march2025_full.csv",
        "games_may2024_cleaned.csv",
        "games_may2024_full.csv",
    ]
    c_dfs = [load_csv_safely(os.path.join(c_path, f)) for f in c_files]
    c_dfs = [clean_columns(df) for df in c_dfs if not df.empty]
    df_c = pd.concat(c_dfs, ignore_index=True)
    df_c["appid"] = df_c["appid"].astype(str)
    logging.info(f"C source combined: {len(df_c)} rows")
    return df_c


# ======== MERGE FUNCTION ========
def merge_sources(a: pd.DataFrame, b: pd.DataFrame, c: pd.DataFrame, columns_to_merge: list[str] = None) -> pd.DataFrame:
    """
    Összefésüli az A, B, C forrásokat AppID alapján.
    C → B → A prioritással tölti ki a hiányzó adatokat.

    Paraméter:
        columns_to_merge: ha meg van adva, csak ezeket az oszlopokat (és appid-t) mergeli.
    """
    logging.info("Merging sources with C→B→A priority...")

    for df in [a, b, c]:
        if not df.empty:
            df["appid"] = df["appid"].astype(str).str.strip()
            df.drop_duplicates(subset="appid", inplace=True)

    if columns_to_merge:
        keep_cols = ["appid"] + [col for col in columns_to_merge if col in a.columns or col in b.columns or col in c.columns]
        a = a[[col for col in keep_cols if col in a.columns]]
        b = b[[col for col in keep_cols if col in b.columns]]
        c = c[[col for col in keep_cols if col in c.columns]]
        logging.info(f"Using subset of columns for merge: {keep_cols}")

    columns = list(dict.fromkeys(
        sum([df.columns.tolist() for df in [a, b, c] if not df.empty], [])
    ))

    all_appids = pd.concat([a[["appid"]], b[["appid"]], c[["appid"]]], ignore_index=True).drop_duplicates()

    D = pd.DataFrame(columns=columns)
    D["appid"] = all_appids["appid"]

    for src in [c, b, a]:
        if not src.empty:
            D = fill_missing_from_source(D, src)

    logging.info(f"Merge complete ({len(D)} rows, {len(columns)} columns)")
    return D



def finalize_sources(D, a, b, c):
    """
    Hozzáad egy 'sources' oszlopot a D (merged_master) DataFrame-hez,
    ami jelzi, hogy a sor melyik eredeti datasetből származik.
    """
    a_ids = set(a["appid"]) if not a.empty else set()
    b_ids = set(b["appid"]) if not b.empty else set()
    c_ids = set(c["appid"]) if not c.empty else set()

    sources = []
    for appid in D["appid"]:
        src = []
        if appid in c_ids:
            src.append("C")
        if appid in b_ids:
            src.append("B")
        if appid in a_ids:
            src.append("A")
        sources.append(",".join(src))

    D["sources"] = sources
    return D

# ======== Segédfüggvények a normalizáláshoz ========
def normalize_screenshots_column(df: pd.DataFrame, source_name: str):
    """
    Kivonatolja a screenshots oszlopot (ha létezik) és visszaadja a thumbnail URL-eket.
    Működik dict/list/str típusokra is.
    """
    thumb_dict = {}

    if "screenshots" not in df.columns:
        return thumb_dict

    for appid, val in df[["appid", "screenshots"]].itertuples(index=False):
        thumb_urls = []

        if val is None:
            continue
        if isinstance(val, float) and np.isnan(val):
            continue

        try:
            data = ast.literal_eval(val) if isinstance(val, str) else val
        except Exception:
            continue

        if isinstance(data, list):
            for item in data:
                if isinstance(item, dict):
                    thumb = item.get("path_thumbnail") or item.get("thumb")
                    if thumb:
                        thumb_urls.append(thumb.strip())
                elif isinstance(item, str):
                    pass

        thumb_urls = [u for u in thumb_urls if isinstance(u, str) and u.startswith("http")]
        thumb_urls = list(dict.fromkeys(thumb_urls))

        thumb_dict[str(appid)] = thumb_urls

    logging.info(f"Normalized thumbnail screenshots for source {source_name} ({len(thumb_dict)} items)")
    return thumb_dict

def process_screenshots(a, b, c):
    """
    Normalizálja a screenshots oszlopokat, 
    visszaadja a thumbnail dict-eket.
    """
    a_thumb = normalize_screenshots_column(a, "A")
    b_thumb = normalize_screenshots_column(b, "B")
    c_thumb = normalize_screenshots_column(c, "C")
    return a_thumb, b_thumb, c_thumb


def normalize_movies_column(df: pd.DataFrame, source_name: str):
    '''
    Normalizálja a 'movies' oszlopot:
    - movies_thumbnail: a 'thumbnail' URL-ek
    - movies_480: a 'webm.480' URL-ek
    - movies_max: a 'webm.max' URL-ek
    '''
    thumb_dict = {}
    m480_dict = {}
    mmax_dict = {}

    if "movies" not in df.columns:
        return thumb_dict, m480_dict, mmax_dict

    for appid, val in df[["appid", "movies"]].itertuples(index=False):
        thumbs = []
        webm_480 = []
        webm_max = []

        if val is None:
            continue
        if isinstance(val, float) and np.isnan(val):
            continue

        try:
            data = ast.literal_eval(val) if isinstance(val, str) else val
        except Exception:
            continue

        if isinstance(data, list):
            for item in data:
                if isinstance(item, dict):
                    t = item.get("thumbnail")
                    if t:
                        thumbs.append(t.strip())
                    w480 = item.get("webm", {}).get("480")
                    if w480:
                        webm_480.append(w480.strip())
                    wmax = item.get("webm", {}).get("max")
                    if wmax:
                        webm_max.append(wmax.strip())

        thumb_dict[str(appid)] = thumbs
        m480_dict[str(appid)] = webm_480
        mmax_dict[str(appid)] = mmax_dict.get(str(appid), []) + mmax_dict.get(str(appid), [])

    logging.info(f"Normalized movies for source {source_name} ({len(thumb_dict)} items)")
    return thumb_dict, m480_dict, mmax_dict

def dedup_join(urls):
    '''
    Egy lista vagy tuple URL-t megtisztít duplikátumoktól és vesszővel összefűzi őket.
    '''
    if not urls or not isinstance(urls, (list, tuple)):
        return ""
    return ", ".join(list(dict.fromkeys(urls)))

def merge_and_finalize(a: pd.DataFrame, b: pd.DataFrame, c: pd.DataFrame, columns_to_merge: list[str] = None) -> pd.DataFrame:
    '''
    Három forrás-DataFrame (A, B, C) egyesítése és véglegesítése.

    - Merge-eli a forrásokat az `appid` alapján.
    - Kategória-, screenshot- és videóadatokat egyesít és átnevez.
    - Thumbnail és 480p videóoszlopokat hoz létre.
    - Eltávolítja a duplikált URL-eket (`dedup_join` segítségével).
    - Összevonja a fejlesztői, kiadói, kategória- és tag-információkat.
    '''
    D = merge_sources(a, b, c, columns_to_merge=columns_to_merge)

    if 'categories' in a.columns:
        D['categories_a'] = D['appid'].map(a.set_index('appid')['categories'])
    if 'categories' in b.columns:
        D['categories_b'] = D['appid'].map(b.set_index('appid')['categories'])
    if 'categories' in c.columns:
        D['categories_c'] = D['appid'].map(c.set_index('appid')['categories'])

    if "screenshots" in D.columns:
        D.rename(columns={"screenshots": "screenshots_full"}, inplace=True)
    a_thumb, b_thumb, c_thumb = process_screenshots(a, b, c)
    D["screenshots_thumb"] = D["appid"].map(
        lambda x: c_thumb.get(x, []) + b_thumb.get(x, []) + a_thumb.get(x, [])
    )

    if "movies" in D.columns:
        D.rename(columns={"movies": "movies_max"}, inplace=True)

    a_thumb_m, a_480, a_max = normalize_movies_column(a, "A")
    b_thumb_m, b_480, b_max = normalize_movies_column(b, "B")
    c_thumb_m, c_480, c_max = normalize_movies_column(c, "C")

    D["movies_thumbnail"] = D["appid"].map(lambda x: c_thumb_m.get(x, []) + b_thumb_m.get(x, []) + a_thumb_m.get(x, []))
    D["movies_480"] = D["appid"].map(lambda x: c_480.get(x, []) + b_480.get(x, []) + a_480.get(x, []))
        

    for col in ["screenshots_thumb", "movies_thumbnail", "movies_480"]:
        D[col] = D[col].apply(dedup_join)

    D = finalize_sources(D, a, b, c)

    D = merge_developers_publishers(D)
    D = merge_categories(D)

    tags_df = merge_tags_column(D, a, b, c)

    tags_collapsed = (
        tags_df.groupby("appid")
        .apply(lambda x: [{"tag_name": t, "weight": w} for t, w in zip(x["tag_name"], x["weight"])])
        .reset_index(name="tags")
    )
        
    D = D.merge(tags_collapsed, on="appid", how="left")

    for col in ["detailed_description", "about_the_game", "short_description"]:
        if col in D.columns:
            D[col] = D[col].astype(str).apply(clean_html_entities)

    if "owners" in D.columns and "estimated_owners" in D.columns:
        D["estimated_owners"] = D["estimated_owners"].combine_first(D["owners"])
        D.drop(columns=["owners"], inplace=True)
    elif "owners" in D.columns:
        D.rename(columns={"owners": "estimated_owners"}, inplace=True)

    if "positive" in D.columns and "positive_ratings" in D.columns:
        D["positive"] = D["positive"].combine_first(D["positive_ratings"])
        D.drop(columns=["positive_ratings"], inplace=True)
    elif "positive_ratings" in D.columns:
        D.rename(columns={"positive_ratings": "positive"}, inplace=True)
    
    if "negative" in D.columns and "negative_ratings" in D.columns:
        D["negative"] = D["negative"].combine_first(D["negative_ratings"])
        D.drop(columns=["negative_ratings"], inplace=True)
    elif "negative_ratings" in D.columns:
        D.rename(columns={"negative_ratings": "negative"}, inplace=True)

    if "average_playtime" in D.columns and "average_playtime_forever" in D.columns:
        D["average_playtime_forever"] = D["average_playtime_forever"].combine_first(D["average_playtime"])
        D.drop(columns=["average_playtime"], inplace=True)
    elif "average_playtime" in D.columns:
        D.rename(columns={"average_playtime": "average_playtime_forever"}, inplace=True)

    if "median_playtime" in D.columns and "median_playtime_forever" in D.columns:
        D["median_playtime_forever"] = D["median_playtime_forever"].combine_first(D["median_playtime"])
        D.drop(columns=["median_playtime"], inplace=True)
    elif "median_playtime" in D.columns:
        D.rename(columns={"median_playtime": "median_playtime_forever"}, inplace=True)

    tag_cols = [c for c in D.columns if c.startswith("tags")]
    
    if len(tag_cols) > 1:
        D["tags"] = D[tag_cols].bfill(axis=1).iloc[:, 0]
        for c in tag_cols:
            if c != "tags":
                D.drop(columns=c, inplace=True, errors="ignore")
        logging.info(f"Combined duplicate tag columns: {tag_cols} → kept unified 'tags'")
    
    if "tags" in D.columns:
        D["tags"] = D["tags"].apply(normalize_tags_column)

    for col in D.columns:
        if "steamspy" in col.lower() and "tag" in col.lower():
            D.drop(columns=[col], inplace=True, errors="ignore")
            logging.info(f"Dropped redundant column: {col}")

    for col in ["supported_languages", "full_audio_languages"]:
        if col in D.columns:
            D[col] = D[col].apply(clean_language_field_merged)
            logging.info(f"Cleaned and normalized language field: {col}")

            
    for col in ["supported_languages", "full_audio_languages"]:
        if col in D.columns:
            before_counts = D[col].apply(len).sum()
            D[col] = D[col].apply(final_clean_language_list)
            after_counts = D[col].apply(len).sum()
            logging.info(
                f"Final language cleanup on {col}: {before_counts} → {after_counts} entries (after filtering)."
            )

    if "supported_languages" in D.columns and "full_audio_languages" in D.columns:
        identical_rows = (D["supported_languages"].astype(str) == D["full_audio_languages"].astype(str)).sum()
        logging.info(f"{identical_rows} rows have identical supported and audio language sets.")

    return D
    
def flatten_values(vals):
    """Lapítja a listákat / stringként tárolt listákat egy sima listává."""
    flat = []
    for v in vals:
        if isinstance(v, str):
            v = v.strip()
            if v.startswith("[") and v.endswith("]"):
                try:
                    sublist = ast.literal_eval(v)
                    if isinstance(sublist, list):
                        flat.extend([str(s).strip() for s in sublist if pd.notna(s)])
                        continue
                except Exception:
                    pass
        flat.append(str(v).strip())
    return list(dict.fromkeys(flat))

def combine_cols(row: pd.Series, cols: list[str]) -> str:
    """
    Több oszlopból származó értékeket kombinál egyetlen, duplikátummentes stringgé.

    - Kinyeri az értékeket a megadott oszlopokból.
    - Támogatja a listákat, NumPy tömböket és skalárokat is.
    - A duplikátumokat eltávolítja és vesszővel elválasztva adja vissza.
    """
    vals = []
    for col in cols:
        val = row.get(col, None)
        if val is None:
            continue
        if isinstance(val, (list, np.ndarray)):
            vals.extend(flatten_values(val))
        else:
            vals.extend(flatten_values([val]))
    return ", ".join(list(dict.fromkeys(vals)))


def merge_developers_publishers(D: pd.DataFrame) -> pd.DataFrame:
    """
    Összevonja a fejlesztői és kiadói oszlopokat, eltávolítva a duplikált neveket.

    - A 'developer' és 'developers' oszlopokból egyesített 'developers' oszlopot hoz létre.
    - A 'publisher' és 'publishers' oszlopokból egyesített 'publishers' oszlopot hoz létre.
    - Az eredeti ('developer', 'publisher') oszlopokat eltávolítja.
    """
    D["developers"] = D.apply(lambda row: combine_cols(row, ["developer", "developers"]), axis=1)
    D["publishers"] = D.apply(lambda row: combine_cols(row, ["publisher", "publishers"]), axis=1)

    for col in ["developer", "publisher"]:
        if col in D.columns:
            D.drop(columns=[col], inplace=True)

    return D

def parse_categories(val) -> list[str]:
    """
    Kategóriaértékek egységes listává alakítása.

    - Kezeli a listákat, NumPy tömböket, stringeket és None értékeket.
    - Tisztítja az üres vagy NaN értékeket.
    - Felismeri a stringként tárolt listákat és a pontosvesszővel tagolt formátumokat.
    """
    if val is None:
        return []
    if isinstance(val, (float, np.floating)) and np.isnan(val):
        return []
    if isinstance(val, (list, np.ndarray)):
        return [str(v).strip() for v in val if isinstance(v, str) and v.strip()]
    if isinstance(val, str):
        val = val.strip()
        if not val:
            return []
        if val.startswith("[") and val.endswith("]"):
            try:
                parsed = ast.literal_eval(val)
                if isinstance(parsed, list):
                    return [str(v).strip() for v in parsed if isinstance(v, str) and v.strip()]
            except Exception:
                pass
        if ";" in val:
            return [v.strip() for v in val.split(";") if v.strip()]
        return [val]
    return []

def combine_categories(row: pd.Series) -> str:
    """
    Egy sor kategóriaoszlopait (A, B, C) kombinálja egyetlen, duplikátummentes stringgé.
    """
    cats_a = parse_categories(row.get("categories_a", row.get("categories", None)))
    cats_b = parse_categories(row.get("categories_b", None))
    cats_c = parse_categories(row.get("categories_c", None))

    merged = []
    seen_lower = set()

    for c in cats_a + cats_b + cats_c:
        cl = c.lower()
        if cl not in seen_lower:
            merged.append(c)
            seen_lower.add(cl)

    return ", ".join(merged)

def merge_categories(D: pd.DataFrame) -> pd.DataFrame:
    """
    A források kategóriaoszlopait egyesíti egységes 'categories' oszlopba.

    - A 'categories_a', 'categories_b', 'categories_c' oszlopokat kombinálja.
    - Duplikátumokat kiszűri, kisbetű-érzéketlen módon.
    - Eltávolítja a felesleges kategóriaoszlopokat.
    """
    category_cols = [c for c in D.columns if "categor" in c.lower()]
    D["categories"] = D.apply(combine_categories, axis=1)

    for col in category_cols:
        if col != "categories":
            D.drop(columns=[col], inplace=True, errors="ignore")

    return D


def merge_tags_column(D: pd.DataFrame, a: pd.DataFrame, b: pd.DataFrame, c: pd.DataFrame) -> pd.DataFrame:
    """
    Egyesíti a címkék (tags) adatokat az A, B, C forrásokból AppID alapján.

    - Az A forrásban a tags stringként tárolt, vesszővel elválasztott lista.
    - A B forrásban a tags dict formátumú (név → súly).
    - A C forrásban vegyes formátumot kezel (stringként tárolt dict is lehet).
    - Az eredmény egy DataFrame, ami appid, tag_name, weight oszlopokat tartalmaz.
    """
    tags_a_dict = {}
    if 'tags' in a.columns:
        for appid, tags_str in zip(a['appid'], a['tags']):
            if isinstance(tags_str, str):
                tags_list = [t.strip() for t in tags_str.split(",") if t.strip()]
                tags_a_dict[str(appid)] = {t: 1 for t in tags_list} 

    tags_b_dict = {}
    if 'tags' in b.columns:
        for appid, tags_json in zip(b['appid'], b['tags']):
            if isinstance(tags_json, dict):
                tags_b_dict[str(appid)] = tags_json

    tags_c_dict = {}
    if 'tags' in c.columns:
        for appid, tags_str in zip(c['appid'], c['tags']):
            if isinstance(tags_str, str):
                try:
                    tags_dict = ast.literal_eval(tags_str)
                    if isinstance(tags_dict, dict):
                        tags_c_dict[str(appid)] = tags_dict
                except:
                    continue

    tag_rows = []
    for appid in D['appid']:
        tag_dict = {}
        tag_dict.update(tags_a_dict.get(str(appid), {}))
        tag_dict.update(tags_b_dict.get(str(appid), {}))
        tag_dict.update(tags_c_dict.get(str(appid), {}))

        for t, w in tag_dict.items():
            tag_rows.append({"appid": appid, "tag_name": t, "weight": w})

    tags_df = pd.DataFrame(tag_rows)
    return tags_df
# =========================================
# VISUALIZATION FUNCTION
# =========================================
def compute_venn_sets(a: pd.DataFrame, b: pd.DataFrame, c: pd.DataFrame):
    """Kiszámítja a halmazokat az appid alapján."""
    set_a = set(a["appid"].astype(str))
    set_b = set(b["appid"].astype(str))
    set_c = set(c["appid"].astype(str))

    only_a = set_a - set_b - set_c
    only_b = set_b - set_a - set_c
    only_c = set_c - set_a - set_b

    a_and_b = (set_a & set_b) - set_c
    a_and_c = (set_a & set_c) - set_b
    b_and_c = (set_b & set_c) - set_a

    all_three = set_a & set_b & set_c

    return {
        "only_a": only_a,
        "only_b": only_b,
        "only_c": only_c,
        "a_and_b": a_and_b,
        "a_and_c": a_and_c,
        "b_and_c": b_and_c,
        "all_three": all_three
    }


def compute_venn_table(a: pd.DataFrame, b: pd.DataFrame, c: pd.DataFrame) -> pd.DataFrame:
    """Visszaadja az elemszámokat egy táblázatban."""
    s = compute_venn_sets(a, b, c)
    data = {
        "csak A": [len(s["only_a"])],
        "csak B": [len(s["only_b"])],
        "csak C": [len(s["only_c"])],
        "A ∩ B": [len(s["a_and_b"])],
        "A ∩ C": [len(s["a_and_c"])],
        "B ∩ C": [len(s["b_and_c"])],
        "A ∩ B ∩ C": [len(s["all_three"])]
    }
    return pd.DataFrame(data)


def plot_and_save_venn(a: pd.DataFrame, b: pd.DataFrame, c: pd.DataFrame, output_dir: str):
    """Létrehozza és elmenti a Venn-diagramot és a táblázatot."""
    s = compute_venn_sets(a, b, c)

    plt.figure(figsize=(8, 8))
    venn3(
        subsets=(
            len(s["only_a"]), len(s["only_b"]), len(s["a_and_b"]),
            len(s["only_c"]), len(s["a_and_c"]), len(s["b_and_c"]),
            len(s["all_three"])
        ),
        set_labels=("Forrás A", "Forrás B", "Forrás C"),
        set_colors=("skyblue", "lightgreen", "lightcoral"),
        alpha=0.8
    )
    plt.title("Adatforrások átfedése – Venn-diagram")
    plt.tight_layout()

    venn_path = os.path.join(output_dir, "venn_diagram.png")
    plt.savefig(venn_path, dpi=300)
    plt.close()
    print(f"Venn-diagram mentve ide: {venn_path}")

    venn_df = compute_venn_table(a, b, c)
    table_path = os.path.join(output_dir, "venn_table.csv")
    venn_df.to_csv(table_path, index=False, encoding="utf-8-sig")
    print(f"Elemszámos Venn-táblázat mentve ide: {table_path}")

    print("\n=== Elemszámos Venn-diagram táblázat ===")
    print(venn_df.to_string(index=False))

# =========================================
# SOURCE SUMMARY TABLE
# =========================================
def save_source_summary(D: pd.DataFrame, output_dir: str):
    """
    Összesítő táblázatot készít arról, hogy hány rekord származik
    csak A-ból, csak B-ből, csak C-ből, illetve ezek kombinációiból.
    Az eredményt CSV-be menti és ki is írja a konzolra.
    """
    if "sources" not in D.columns:
        logging.warning("A 'sources' oszlop nem található a merged táblában.")
        return

    summary = D["sources"].value_counts().reset_index()
    summary.columns = ["forrás_kombináció", "rekordok_száma"]

    summary["tartalmaz_A"] = summary["forrás_kombináció"].str.contains("A", regex=False)
    summary["tartalmaz_B"] = summary["forrás_kombináció"].str.contains("B", regex=False)
    summary["tartalmaz_C"] = summary["forrás_kombináció"].str.contains("C", regex=False)

    output_file = os.path.join(output_dir, "source_summary.csv")
    summary.to_csv(output_file, index=False, encoding="utf-8-sig")

    logging.info(f"Source summary saved: {output_file}")
    print("\n=== Forrásonkénti rekordösszesítő táblázat ===")
    print(summary.to_string(index=False))

def validate_integrity(D: pd.DataFrame, output_dir: str):
    """
    Adatintegritás-ellenőrzés: duplikált appid, hiányzó értékek, típushibák stb.
    Eredményt logolja és CSV-be menti.
    """
    results = []

    dup_count = D["appid"].duplicated().sum()
    results.append({"ellenőrzés": "Duplikált appid-ek", "hibák_száma": dup_count})

    na_appid = D["appid"].isna().sum()
    results.append({"ellenőrzés": "Hiányzó appid-ek", "hibák_száma": na_appid})

    if "name" in D.columns:
        na_name = D["name"].isna().sum()
        results.append({"ellenőrzés": "Hiányzó játéknevek", "hibák_száma": na_name})

    if "sources" in D.columns:
        na_sources = (D["sources"] == "").sum()
        results.append({"ellenőrzés": "Hiányzó forrásjelölés", "hibák_száma": na_sources})

    if "release_date" in D.columns:
        invalid_dates = pd.to_datetime(D["release_date"], errors="coerce").isna().sum()
        results.append({"ellenőrzés": "Érvénytelen release_date", "hibák_száma": invalid_dates})

    integrity_df = pd.DataFrame(results)
    output_file = os.path.join(output_dir, "integrity_report.csv")
    integrity_df.to_csv(output_file, index=False, encoding="utf-8-sig")

    logging.info(f"Integrity check completed, saved to {output_file}")
    print("\n=== Integritás ellenőrzési összesítő ===")
    print(integrity_df.to_string(index=False))


def save_merged(D, path):
    """
    Elmenti az egyesített (merged_master) táblát a megadott könyvtárba UTF-8 kódolással.
    """
    output_file = os.path.join(path, "merged_master.csv")
    D.to_csv(output_file, index=False, encoding="utf-8-sig")
    return output_file


# ======== MAIN ========
def main():
    warnings.filterwarnings("ignore", category=FutureWarning)
    logging.info("=== Starting merge process ===")
    a = load_source_a(A_PATH)
    a_output_file = os.path.join(OUTPUT_PATH, "A_merged.csv")
    a.to_csv(a_output_file, index=False, encoding="utf-8")

    b = load_source_b(B_PATH)

    b.to_csv(os.path.join(OUTPUT_PATH, "B_full.csv"), index=False, encoding="utf-8")
    logging.info("B full DataFrame written to CSV")
    
    c = load_source_c(C_PATH)

    plot_and_save_venn(a, b, c, OUTPUT_PATH)

    D = merge_and_finalize(a, b, c)

    #D = merge_and_finalize(a, b, c, columns_to_merge = ["owners"])

    output_file = save_merged(D, OUTPUT_PATH)
    save_source_summary(D, OUTPUT_PATH)
    validate_integrity(D, OUTPUT_PATH)
    logging.info(f"Merged master table saved to: {output_file}")
    
    logging.info("=== Merge process successfully completed ===")

if __name__ == "__main__":
    main()

[2025-11-09 11:41:09] INFO: === Starting merge process ===
[2025-11-09 11:41:09] INFO: Loaded: steam.csv (27075 rows)
[2025-11-09 11:41:10] INFO: Loaded: steam_description_data_cleaned.csv (27334 rows)
[2025-11-09 11:41:11] INFO: Loaded: steam_media_data.csv (27332 rows)
[2025-11-09 11:41:11] INFO: Loaded: steam_support_info.csv (27136 rows)
[2025-11-09 11:41:12] INFO: Loaded: steamspy_tag_data.csv (29022 rows)
[2025-11-09 11:41:12] INFO: Loaded: steam_requirements_data.csv (27319 rows)
[2025-11-09 11:41:14] INFO: SteamSpy tags converted → 28447 appid with tag data
[2025-11-09 11:41:15] INFO: A source merged: 27075 rows
[2025-11-09 11:41:34] INFO: B source loaded from JSON: 111452 rows
[2025-11-09 11:41:47] INFO: B full DataFrame written to CSV
[2025-11-09 11:41:54] INFO: Loaded: games_march2025_cleaned.csv (89618 rows)
[2025-11-09 11:42:00] INFO: Loaded: games_march2025_full.csv (94948 rows)
[2025-11-09 11:42:05] INFO: Loaded: games_may2024_cleaned.csv (83646 rows)
[2025-11-09 11:42:1

Venn-diagram mentve ide: C:\Users\zalma\merge\venn_diagram.png


[2025-11-09 11:42:11] INFO: Merging sources with C→B→A priority...


Elemszámos Venn-táblázat mentve ide: C:\Users\zalma\merge\venn_table.csv

=== Elemszámos Venn-diagram táblázat ===
 csak A  csak B  csak C  A ∩ B  A ∩ C  B ∩ C  A ∩ B ∩ C
   1234    5731     161   1400      8  79888      24433


[2025-11-09 11:42:32] INFO: Merge complete (112855 rows, 63 columns)
[2025-11-09 11:42:37] INFO: Normalized thumbnail screenshots for source A (27075 items)
[2025-11-09 11:42:37] INFO: Normalized thumbnail screenshots for source B (111452 items)
[2025-11-09 11:42:42] INFO: Normalized thumbnail screenshots for source C (104490 items)
[2025-11-09 11:42:44] INFO: Normalized movies for source A (25393 items)
[2025-11-09 11:42:45] INFO: Normalized movies for source B (111452 items)
[2025-11-09 11:42:48] INFO: Normalized movies for source C (104490 items)
[2025-11-09 11:44:12] INFO: Combined duplicate tag columns: ['tags_x', 'tags_y'] → kept unified 'tags'
[2025-11-09 11:44:25] INFO: Dropped redundant column: steamspy_tags
[2025-11-09 11:44:33] INFO: Cleaned and normalized language field: supported_languages
[2025-11-09 11:44:39] INFO: Cleaned and normalized language field: full_audio_languages
[2025-11-09 11:44:48] INFO: Final language cleanup on supported_languages: 542206 → 542160 entries


=== Forrásonkénti rekordösszesítő táblázat ===
forrás_kombináció  rekordok_száma  tartalmaz_A  tartalmaz_B  tartalmaz_C
              C,B           79888        False         True         True
            C,B,A           24433         True         True         True
                B            5731        False         True        False
              B,A            1400         True         True        False
                A            1234         True        False        False
                C             161        False        False         True
              C,A               8         True        False         True

=== Integritás ellenőrzési összesítő ===
              ellenőrzés  hibák_száma
      Duplikált appid-ek            0
        Hiányzó appid-ek            0
      Hiányzó játéknevek            0
   Hiányzó forrásjelölés            0
Érvénytelen release_date            8


In [11]:
import os
import csv
import ast
import pandas as pd
import numpy as np
import logging
import json
import re
from bs4 import BeautifulSoup
from typing import Any
import matplotlib.pyplot as plt
import warnings
import sys

# ======== PATHS ========
BASE_PATH = r"C:\Users\zalma"
D_PATH = os.path.join(BASE_PATH, "merge")
OUTPUT_PATH = os.path.join(BASE_PATH, "split")

# ======== LOGGING CONFIGURATION ========
os.makedirs(OUTPUT_PATH, exist_ok=True)
LOG_FILE = os.path.join(OUTPUT_PATH, "merge_log.txt")

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] %(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.FileHandler(LOG_FILE, encoding="utf-8"), logging.StreamHandler()],
)

# ======== HELPER FUNCTIONS ========
def load_csv_safely(path: str, **kwargs: Any) -> pd.DataFrame:
    """
    Betölt egy CSV fájlt, hiba esetén üres DataFrame-et ad vissza.
    """
    try:
        df = pd.read_csv(path, **kwargs)
        logging.info(f"Loaded: {os.path.basename(path)} ({len(df)} rows)")
        return df
    except Exception as e:
        logging.error(f"Error loading {path}: {e}")
        return pd.DataFrame()


# ======== Segédfüggvények a splittelt táblákhoz ========
def create_media_table(master_df: pd.DataFrame, output_dir: str = None) -> pd.DataFrame:
    """
    Létrehozza a media táblát a merged_master-ből.
    
    """
    media_cols = ["appid", "header_image", "background"]
    media_df = master_df[[c for c in media_cols if c in master_df.columns]].copy()
    
    media_df = media_df.dropna(subset=["header_image"]).reset_index(drop=True)
    
    media_df.insert(0, "mediaid", range(1, len(media_df)+1))
    
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        path = os.path.join(output_dir, "media.csv")
        media_df.to_csv(path, index=False)
        logging.info(f"Saved 'media.csv' ({len(media_df)} rows) to {output_dir}")
    
    return media_df

def join_urls(x) -> str:
    """
    Lista vagy string URL-eket egységes, vesszővel elválasztott stringgé alakít.

    - Ha lista, akkor elemeit összefűzi ', ' elválasztóval.
    - Ha már string, változatlanul visszaadja.
    - Egyéb esetben üres stringet ad vissza.
    """
    if isinstance(x, list):
        return ", ".join(x)
    elif isinstance(x, str):
        return x
    return ""


def create_screenshots_table(master_df: pd.DataFrame, output_dir: str = None) -> pd.DataFrame:
    """
    Létrehozza a screenshots táblát a master DataFrame-ből.

    - Kiválasztja az 'appid', 'screenshots_full' és 'screenshots_thumb' oszlopokat.
    - A listákat stringgé alakítja (`join_urls` segítségével).
    - Eltávolítja az üres sorokat.
    - Hozzáad egy automatikus 'screenshotid' azonosítót.
    - CSV-fájlba menti az eredményt.
    """
    cols = ["appid"]
    for c in ["screenshots_full", "screenshots_thumb"]:
        if c in master_df.columns:
            cols.append(c)

    df = master_df[cols].copy()

    for c in ["screenshots_full", "screenshots_thumb"]:
        if c in df.columns:
            df[c] = df[c].apply(join_urls)

    df = df[
        (df.get("screenshots_full", "") != "") |
        (df.get("screenshots_thumb", "") != "")
    ].reset_index(drop=True)

    df.insert(0, "screenshotid", range(1, len(df) + 1))

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        path = os.path.join(output_dir, "screenshots.csv")
        df.to_csv(path, index=False)
        logging.info(f"Saved 'screenshots.csv' ({len(df)} rows) to {output_dir}")

    return df


def create_movies_table(master_df: pd.DataFrame, output_dir: str = None) -> pd.DataFrame:
    """
    Létrehozza a 'movies' táblát a master DataFrame-ből.

    - Kiválasztja az 'appid', 'movies_thumbnail', 'movies_max' és 'movies_480' oszlopokat.
    - A listákat stringgé alakítja (`join_urls` segítségével).
    - Csak azokat a sorokat tartja meg, ahol legalább egy URL szerepel.
    - Hozzáad egy automatikus 'movieid' azonosítót.
    - (Opcionálisan) CSV-fájlba menti az eredményt.

    Visszatér: a videókat tartalmazó DataFrame.
    """
    cols = ["appid"]
    for c in ["movies_thumbnail", "movies_max", "movies_480"]:
        if c in master_df.columns:
            cols.append(c)

    df = master_df[cols].copy()

    for c in ["movies_thumbnail", "movies_max", "movies_480"]:
        if c in df.columns:
            df[c] = df[c].apply(join_urls)

    df = df[
        (df.get("movies_thumbnail", "") != "") |
        (df.get("movies_max", "") != "") |
        (df.get("movies_480", "") != "")
    ].reset_index(drop=True)

    df.insert(0, "movieid", range(1, len(df) + 1))

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        path = os.path.join(output_dir, "movies.csv")
        df.to_csv(path, index=False)
        logging.info(f"Saved 'movies.csv' ({len(df)} rows) to {output_dir}")

    return df

def create_support_table(master_df: pd.DataFrame, output_dir: str = None) -> pd.DataFrame:
    """
    Létrehozza a support táblát a merged_master-ből.
    Tartalmazza:
      - supportid (1-től generált)
      - appid
      - support_url
      - support_email
    """
    cols = ["appid"]
    for c in ["support_url", "support_email"]:
        if c in master_df.columns:
            cols.append(c)

    df = master_df[cols].copy()

    for c in ["support_url", "support_email"]:
        if c in df.columns:
            df[c] = df[c].fillna("").astype(str)

    df = df[(df.get("support_url", "") != "") | (df.get("support_email", "") != "")].reset_index(drop=True)

    df.insert(0, "supportid", range(1, len(df) + 1))

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        path = os.path.join(output_dir, "support.csv")
        df.to_csv(path, index=False)
        logging.info(f"Saved 'support.csv' ({len(df)} rows) to {output_dir}")

    return df

def clean_requirements_text(text):
    if not text or pd.isna(text):
        return ""
    soup = BeautifulSoup(str(text), "html.parser")

    for br in soup.find_all("br"):
        br.replace_with(" ")

    for li in soup.find_all("li"):
        li.replace_with(f"{li.get_text()}, ")

    cleaned = soup.get_text(separator=" ").strip()

    cleaned = re.sub(r'\s+', ' ', cleaned)

    cleaned = re.sub(r',\s*$', '', cleaned)

    cleaned = re.sub(r'^[\)\("\'\s,]+', '', cleaned)

    cleaned = re.sub(r'(?i)^(minimum|recommended)[:\s-]*', '', cleaned).strip()

    return cleaned


def split_min_rec(text):
    """
    Szétválasztja a minimum és recommended részt a stringből.
    Kis-/nagybetűt normalizál, ha a minimumban benne van a recommended, szétvágja.
    """
    if not text or pd.isna(text):
        return "", ""
    text = str(text).strip()
    parts = re.split(r"(?i)Recommended[:\s]*", text, maxsplit=1)
    min_part = parts[0].strip() if parts else ""
    rec_part = parts[1].strip() if len(parts) > 1 else ""
    return min_part, rec_part


def create_requirements_table(master_df: pd.DataFrame, output_dir: str = None) -> pd.DataFrame:
    rows = []

    for _, r in master_df.iterrows():
        appid = r['appid']

        # --- Windows (pc_requirements) ---
        pc_val = r.get('pc_requirements', "")
        if pd.notna(pc_val) and str(pc_val).strip():
            try:
                val_dict = ast.literal_eval(pc_val)
                win_min = clean_requirements_text(val_dict.get('minimum', ""))
                win_rec = clean_requirements_text(val_dict.get('recommended', ""))
            except Exception:
                text = clean_requirements_text(pc_val)
                win_min, win_rec = split_min_rec(text)

            if win_min:
                win_min, extra_rec = split_min_rec(win_min)
                if win_min:
                    rows.append({'appid': appid, 'os': 'windows', 'type': 'minimum', 'requirements': win_min})
                if extra_rec:
                    rows.append({'appid': appid, 'os': 'windows', 'type': 'recommended', 'requirements': extra_rec})

            if win_rec:
                rows.append({'appid': appid, 'os': 'windows', 'type': 'recommended', 'requirements': win_rec})


        # --- Mac ---
        val = r.get('mac_requirements', "")
        if pd.notna(val):
            val_str = str(val).strip()
            if val_str and val_str not in ["[]", "{}", "nan", "None"]:
                try:
                    val_dict = ast.literal_eval(val)
                    min_val = val_dict.get('minimum', "")
                    rec_val = val_dict.get('recommended', "")
                except Exception:
                    min_val = val
                    rec_val = ""

                min_val = clean_requirements_text(min_val)
                rec_val = clean_requirements_text(rec_val)

                if min_val:
                    min_val, extra_rec = split_min_rec(min_val)
                    if min_val:
                        rows.append({'appid': appid, 'os': 'mac', 'type': 'minimum', 'requirements': min_val})
                    if extra_rec:
                        rows.append({'appid': appid, 'os': 'mac', 'type': 'recommended', 'requirements': extra_rec})
                if rec_val:
                    rows.append({'appid': appid, 'os': 'mac', 'type': 'recommended', 'requirements': rec_val})

        # --- Linux ---
        val = r.get('linux_requirements', "")
        if pd.notna(val):
            val_str = str(val).strip()
            if val_str and val_str not in ["[]", "{}", "nan", "None"]:
                try:
                    val_dict = ast.literal_eval(val)
                    min_val = val_dict.get('minimum', "")
                    rec_val = val_dict.get('recommended', "")
                except Exception:
                    min_val = val
                    rec_val = ""

                min_val = clean_requirements_text(min_val)
                rec_val = clean_requirements_text(rec_val)

                if min_val:
                    min_val, extra_rec = split_min_rec(min_val)
                    if min_val:
                        rows.append({'appid': appid, 'os': 'linux', 'type': 'minimum', 'requirements': min_val})
                    if extra_rec:
                        rows.append({'appid': appid, 'os': 'linux', 'type': 'recommended', 'requirements': extra_rec})
                if rec_val:
                    rows.append({'appid': appid, 'os': 'linux', 'type': 'recommended', 'requirements': rec_val})



    df_req = pd.DataFrame(rows)
    if not df_req.empty:
        df_req.insert(0, 'reqid', range(1, len(df_req)+1))

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        path = os.path.join(output_dir, "requirements.csv")
        df_req.to_csv(path, index=False)
        logging.info(f"Saved 'requirements.csv' ({len(df_req)} rows) to {output_dir}")

    return df_req


def create_genres_table(master_df: pd.DataFrame, output_dir: str = None):
    """
    Létrehozza a game_genre és genres táblákat:
    - game_genre: appid + genreid
    - genres: genreid + genre_name (eredeti genres mező)
    """
    rows = []

    for _, row in master_df.iterrows():
        appid = row["appid"]
        genres_raw = row.get("genres", "")

        text = str(genres_raw).strip()
        if text in ["", "[]", "['']"]:
            continue

        rows.append({"appid": appid, "genre_name": text})

    df_flat = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)

    df_flat.insert(1, "genreid", range(1, len(df_flat)+1))

    game_genre_df = df_flat[['appid', 'genreid']].copy()
    genres_df = df_flat[['genreid', 'genre_name']].copy()

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        genres_path = os.path.join(output_dir, "genres.csv")
        game_genre_path = os.path.join(output_dir, "game_genre.csv")
        genres_df.to_csv(genres_path, index=False)
        game_genre_df.to_csv(game_genre_path, index=False)
        logging.info(f"Saved 'genres.csv' ({len(genres_df)} rows) to {output_dir}")
        logging.info(f"Saved 'game_genre.csv' ({len(game_genre_df)} rows) to {output_dir}")

    return genres_df, game_genre_df

def create_platforms_table(master_df: pd.DataFrame, output_dir: str = None):
    """
    Létrehozza a game_platform és platforms táblákat:
    - game_platform: appid + platid
    - platforms: platid + windows/linux/mac logikai mezők
    """
    rows = []

    for _, row in master_df.iterrows():
        appid = row["appid"]
        windows = bool(row.get("windows", False))
        linux = bool(row.get("linux", False))
        mac = bool(row.get("mac", False))

        rows.append({
            "appid": appid,
            "windows": windows,
            "linux": linux,
            "mac": mac
        })

    df_flat = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)

    df_flat.insert(1, "platid", range(1, len(df_flat)+1))

    game_platform_df = df_flat[['appid', 'platid']].copy()
    platforms_df = df_flat[['platid', 'windows', 'linux', 'mac']].copy()

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        platforms_path = os.path.join(output_dir, "platforms.csv")
        game_platform_path = os.path.join(output_dir, "game_platform.csv")
        platforms_df.to_csv(platforms_path, index=False)
        game_platform_df.to_csv(game_platform_path, index=False)
        logging.info(f"Saved 'platforms.csv' ({len(platforms_df)} rows) to {output_dir}")
        logging.info(f"Saved 'game_platform.csv' ({len(game_platform_df)} rows) to {output_dir}")

    return platforms_df, game_platform_df

def create_packages_table(master_df: pd.DataFrame, output_dir: str):
    rows_game_package = []
    rows_packages = []
    rows_sub_package = []

    packid_counter = 1

    for _, row in master_df.iterrows():
        appid = row["appid"]
        packages_raw = row.get("packages", "")

        if pd.isna(packages_raw) or not str(packages_raw).strip():
            continue

        try:
            packages_list = ast.literal_eval(packages_raw)
        except Exception:
            continue

        if not isinstance(packages_list, list):
            continue

        for pkg in packages_list:
            title = pkg.get("title", "").strip()
            description = pkg.get("description", "").strip()

            if not title:
                continue

            rows_game_package.append({"appid": appid, "packid": packid_counter})

            rows_packages.append({"packid": packid_counter, "title": title, "description": description})

            subs = pkg.get("subs", [])
            for sub in subs:
                sub_text = sub.get("text", "").strip()
                price = sub.get("price", None)
                rows_sub_package.append({"packid": packid_counter, "sub_text": sub_text, "price": price})

            packid_counter += 1

    df_game_package = pd.DataFrame(rows_game_package)
    df_packages = pd.DataFrame(rows_packages)
    df_sub_package = pd.DataFrame(rows_sub_package)

    os.makedirs(output_dir, exist_ok=True)
    df_game_package.to_csv(os.path.join(output_dir, "game_package.csv"), index=False)
    df_packages.to_csv(os.path.join(output_dir, "packages.csv"), index=False)
    df_sub_package.to_csv(os.path.join(output_dir, "sub_package.csv"), index=False)

    logging.info(f"Saved game_package.csv ({len(df_game_package)} rows)")
    logging.info(f"Saved packages.csv ({len(df_packages)} rows)")
    logging.info(f"Saved sub_package.csv ({len(df_sub_package)} rows)")

    return df_game_package, df_packages, df_sub_package

def create_developer_tables(master_df: pd.DataFrame, output_dir: str = None):
    """
    Létrehozza a developers és game_developer táblákat úgy,
    hogy minden játékhoz egy sor tartozik, még ha több fejlesztője is van.
    - game_developer: appid + devid (1-től generált)
    - developers: devid + name (összefűzött fejlesztők)
    """
    rows = []

    for _, row in master_df.iterrows():
        appid = row["appid"]
        devs_raw = row.get("developers", "")
        if not devs_raw or pd.isna(devs_raw):
            continue
        
        if isinstance(devs_raw, list):
            dev_list = [str(d).strip() for d in devs_raw if str(d).strip()]
        else:
            dev_list = [d.strip() for d in str(devs_raw).split(",") if d.strip()]

        if not dev_list:
            continue
        
        combined_devs = ", ".join(dev_list)
        rows.append({"appid": appid, "developer_name": combined_devs})

    df_flat = pd.DataFrame(rows).reset_index(drop=True)

    df_flat.insert(1, "devid", range(1, len(df_flat)+1))

    game_developer_df = df_flat[['appid', 'devid']].copy()
    developers_df = df_flat[['devid', 'developer_name']].copy()

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        developers_path = os.path.join(output_dir, "developers.csv")
        game_developer_path = os.path.join(output_dir, "game_developer.csv")
        developers_df.to_csv(developers_path, index=False)
        game_developer_df.to_csv(game_developer_path, index=False)
        logging.info(f"Saved 'developers.csv' ({len(developers_df)} rows) to {output_dir}")
        logging.info(f"Saved 'game_developer.csv' ({len(game_developer_df)} rows) to {output_dir}")

    return developers_df, game_developer_df

def create_publisher_tables(master_df: pd.DataFrame, output_dir: str = None):
    """
    Létrehozza a publishers és game_publisher táblákat úgy,
    hogy minden játékhoz egy sor tartozik, még ha több kiadója is van.
    - game_publisher: appid + pubid (1-től generált)
    - publishers: pubid + name (összefűzött kiadók)
    """
    rows = []

    for _, row in master_df.iterrows():
        appid = row["appid"]
        pubs_raw = row.get("publishers", "")
        if not pubs_raw or pd.isna(pubs_raw):
            continue
        
        if isinstance(pubs_raw, list):
            pub_list = [str(p).strip() for p in pubs_raw if str(p).strip()]
        else:
            pub_list = [p.strip() for p in str(pubs_raw).split(",") if p.strip()]

        if not pub_list:
            continue
        
        combined_pubs = ", ".join(pub_list)
        rows.append({"appid": appid, "publisher_name": combined_pubs})

    df_flat = pd.DataFrame(rows).reset_index(drop=True)

    df_flat.insert(1, "pubid", range(1, len(df_flat)+1))

    game_publisher_df = df_flat[['appid', 'pubid']].copy()
    publishers_df = df_flat[['pubid', 'publisher_name']].copy()

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        publishers_path = os.path.join(output_dir, "publishers.csv")
        game_publisher_path = os.path.join(output_dir, "game_publisher.csv")
        publishers_df.to_csv(publishers_path, index=False)
        game_publisher_df.to_csv(game_publisher_path, index=False)
        logging.info(f"Saved 'publishers.csv' ({len(publishers_df)} rows) to {output_dir}")
        logging.info(f"Saved 'game_publisher.csv' ({len(game_publisher_df)} rows) to {output_dir}")

    return publishers_df, game_publisher_df

def create_categories_table(master_df: pd.DataFrame, output_dir: str = None):
    """
    Létrehozza a game_category és categories táblákat:
    - game_category: appid + catid
    - categories: catid + name (eredeti categories mező)
    """
    rows = []

    for _, row in master_df.iterrows():
        appid = row["appid"]
        categories_raw = row.get("categories", "")

        text = str(categories_raw).strip()
        if text in ["", "[]", "['']"]:
            continue

        rows.append({"appid": appid, "name": text})

    df_flat = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)

    df_flat.insert(1, "catid", range(1, len(df_flat)+1))

    game_category_df = df_flat[['appid', 'catid']].copy()
    categories_df = df_flat[['catid', 'name']].copy()

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        categories_path = os.path.join(output_dir, "categories.csv")
        game_category_path = os.path.join(output_dir, "game_category.csv")
        categories_df.to_csv(categories_path, index=False)
        game_category_df.to_csv(game_category_path, index=False)
        logging.info(f"Saved 'categories.csv' ({len(categories_df)} rows) to {output_dir}")
        logging.info(f"Saved 'game_category.csv' ({len(game_category_df)} rows) to {output_dir}")

    return categories_df, game_category_df


def create_tags_table(master_df: pd.DataFrame, output_dir: str = None):
    """
    Egyszerűsített tags táblageneráló függvény.
    A merged_master.csv 'tags' oszlopából két táblát készít:
      - game_tag.csv: appid–tagid kapcsolatok
      - tags.csv: tagid–tag_name–weight lista
    """
    rows_game_tag = []
    rows_tags = []
    tagid_counter = 1

    for _, row in master_df.iterrows():
        appid = row["appid"]
        tags_val = row.get("tags")

        if not tags_val or pd.isna(tags_val):
            continue

        if isinstance(tags_val, str):
            try:
                tags_val = ast.literal_eval(tags_val)
            except Exception:
                continue

        if not isinstance(tags_val, list):
            continue

        for tag_entry in tags_val:
            if not isinstance(tag_entry, dict):
                continue
            tag_name = tag_entry.get("tag_name")
            weight = tag_entry.get("weight", 1)

            if not tag_name:
                continue

            rows_game_tag.append({"appid": appid, "tagid": tagid_counter})
            rows_tags.append({"tagid": tagid_counter, "tag_name": tag_name, "weight": weight})
            tagid_counter += 1

    game_tag_df = pd.DataFrame(rows_game_tag)
    tags_df = pd.DataFrame(rows_tags)

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        game_tag_path = os.path.join(output_dir, "game_tag.csv")
        tags_path = os.path.join(output_dir, "tags.csv")

        game_tag_df.to_csv(game_tag_path, index=False, encoding="utf-8-sig")
        tags_df.to_csv(tags_path, index=False, encoding="utf-8-sig")

        logging.info(f"Saved 'game_tag.csv' ({len(game_tag_df)} rows) to {output_dir}")
        logging.info(f"Saved 'tags.csv' ({len(tags_df)} rows) to {output_dir}")

    return game_tag_df, tags_df

def create_languages_table(master_df: pd.DataFrame, output_dir: str = None):
    """
    A master_df 'supported_languages' és 'full_audio_languages' oszlopaiból
    három táblát hoz létre a relációs séma szerint:

    1. languages.csv             → id | name
    2. game_subtitles.csv        → appid | languageid  (feliratok)
    3. game_audio_language.csv   → appid | languageid  (hang)
    """
    lang_name_to_id = {}
    next_lang_id = 1

    rows_languages = []
    rows_game_subtitles = []
    rows_game_audio = []

    for _, row in master_df.iterrows():
        appid = row["appid"]

        supported_raw = row.get("supported_languages", "")
        supported = []
        if pd.notna(supported_raw) and str(supported_raw).strip():
            try:
                val = ast.literal_eval(str(supported_raw))
                if isinstance(val, list):
                    supported = [v.strip() for v in val if isinstance(v, str) and v.strip()]
                elif isinstance(val, str):
                    supported = [v.strip() for v in val.split(",") if v.strip()]
            except Exception:
                supported = [v.strip() for v in str(supported_raw).split(",") if v.strip()]

        audio_raw = row.get("full_audio_languages", "")
        full_audio = []
        if pd.notna(audio_raw) and str(audio_raw).strip():
            try:
                val = ast.literal_eval(str(audio_raw))
                if isinstance(val, list):
                    full_audio = [v.strip() for v in val if isinstance(v, str) and v.strip()]
                elif isinstance(val, str):
                    full_audio = [v.strip() for v in val.split(",") if v.strip()]
            except Exception:
                full_audio = [v.strip() for v in str(audio_raw).split(",") if v.strip()]

        all_langs = set(supported + full_audio)

        for lang in all_langs:
            if lang not in lang_name_to_id:
                lang_name_to_id[lang] = next_lang_id
                rows_languages.append({"id": next_lang_id, "name": lang})
                next_lang_id += 1

            langid = lang_name_to_id[lang]

            if lang in supported:
                rows_game_subtitles.append({"appid": appid, "languageid": langid})
            if lang in full_audio:
                rows_game_audio.append({"appid": appid, "languageid": langid})

    languages_df = pd.DataFrame(rows_languages)
    game_subtitles_df = pd.DataFrame(rows_game_subtitles)
    game_audio_df = pd.DataFrame(rows_game_audio)

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        languages_df.to_csv(os.path.join(output_dir, "languages.csv"), index=False, encoding="utf-8-sig")
        game_subtitles_df.to_csv(os.path.join(output_dir, "game_subtitles.csv"), index=False, encoding="utf-8-sig")
        game_audio_df.to_csv(os.path.join(output_dir, "game_audio_language.csv"), index=False, encoding="utf-8-sig")

        logging.info(f"Saved languages ({len(languages_df)}), "
                     f"game_subtitles ({len(game_subtitles_df)}), "
                     f"game_audio_language ({len(game_audio_df)}) to {output_dir}")

    return languages_df, game_subtitles_df, game_audio_df


def create_description_table(master_df: pd.DataFrame, output_dir: str = None) -> pd.DataFrame:
    """
    Létrehozza a 'description' táblát a master DataFrame-ből.

    Tartalmazza:
      - descriptionid (1-től generált)
      - appid
      - detailed_description
      - about_the_game
      - short_description
    """
    cols = ["appid", "detailed_description", "about_the_game", "short_description"]
    existing_cols = [c for c in cols if c in master_df.columns]

    if not existing_cols:
        logging.warning("No description columns found in master dataframe.")
        return pd.DataFrame()

    df = master_df[existing_cols].copy()

    df = df[
        (df.get("detailed_description", "") != "") |
        (df.get("about_the_game", "") != "") |
        (df.get("short_description", "") != "")
    ].reset_index(drop=True)

    df.insert(0, "descriptionid", range(1, len(df) + 1))

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        path = os.path.join(output_dir, "description.csv")
        df.to_csv(path, index=False)
        logging.info(f"Saved 'description.csv' ({len(df)} rows) to {output_dir}")

    return df

def create_game_table(master_df: pd.DataFrame, output_dir: str = None) -> pd.DataFrame:
    """
    Létrehozza a 'game.csv' táblát a master DataFrame-ből.
    Csak az appid, name és release_date mezőket tartalmazza.
    Nem szűri ki az üres neveket.
    """
    cols = ["appid", "name", "release_date","estimated_owners","required_age","price","dlc_count","recommendations","notes",
           "website","metacritic_score","metacritic_url","achievements","user_score","score_rank","positive","negative",
           "average_playtime_forever","average_playtime_2weeks","median_playtime_forever","median_playtime_2weeks",
           "peak_ccu","discount","pct_pos_total","pct_pos_recent","num_reviews_total","num_reviews_recent","reviews","english"]
    existing_cols = [c for c in cols if c in master_df.columns]

    if "appid" not in existing_cols:
        logging.warning("Missing 'appid' column in master dataframe.")
        return pd.DataFrame()

    df = master_df[existing_cols].copy()

    df = df.reset_index(drop=True)

    if "recommendations" in df.columns:
        df.rename(columns={"recommendations": "num_recommendations"}, inplace=True)

    if "achievements" in df.columns:
        df.rename(columns={"achievements": "num_achievements"}, inplace=True)

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        path = os.path.join(output_dir, "game.csv")
        df.to_csv(path, index=False, encoding="utf-8-sig")
        logging.info(f"Saved 'game.csv' ({len(df)} rows) to {output_dir}")

    return df


def main():
    warnings.filterwarnings("ignore", category=FutureWarning)
    logging.info("=== Starting splitting process ===")
    D = load_csv_safely(os.path.join(D_PATH, "merged_master.csv"))
    
    media_df = create_media_table(D, output_dir=OUTPUT_PATH)
    screenshot_df = create_screenshots_table(D, output_dir=OUTPUT_PATH)
    movies_df = create_movies_table(D, output_dir=OUTPUT_PATH)
    support_df = create_support_table(D, output_dir=OUTPUT_PATH)
    requirements_df = create_requirements_table(D, output_dir=OUTPUT_PATH)
    platforms_df = create_platforms_table(D, output_dir=OUTPUT_PATH)
    packages_df = create_packages_table(D, output_dir=OUTPUT_PATH)
    developer_df = create_developer_tables(D, output_dir=OUTPUT_PATH)
    publisher_df = create_publisher_tables(D, output_dir=OUTPUT_PATH)
    genres_df = create_genres_table(D, output_dir=OUTPUT_PATH)
    categories_df = create_categories_table(D, output_dir=OUTPUT_PATH)
    tags_df = create_tags_table(D, output_dir=OUTPUT_PATH)
    description_df = create_description_table(D, output_dir=OUTPUT_PATH)
    game_df = create_game_table(D, output_dir=OUTPUT_PATH)
    languages_df = create_languages_table(D, output_dir=OUTPUT_PATH)
    
    
if __name__ == "__main__":
    main()

[2025-11-09 11:46:03] INFO: === Starting splitting process ===
C:\Users\zalma\AppData\Local\Temp\ipykernel_11852\1140561955.py:37: DtypeWarning: Columns (4,15,20,21,22,23,24,28,29,30,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, **kwargs)
[2025-11-09 11:46:11] INFO: Loaded: merged_master.csv (112855 rows)
[2025-11-09 11:46:11] INFO: Saved 'media.csv' (112855 rows) to C:\Users\zalma\split
[2025-11-09 11:46:15] INFO: Saved 'screenshots.csv' (112855 rows) to C:\Users\zalma\split
[2025-11-09 11:46:16] INFO: Saved 'movies.csv' (112771 rows) to C:\Users\zalma\split
[2025-11-09 11:46:16] INFO: Saved 'support.csv' (105409 rows) to C:\Users\zalma\split
[2025-11-09 11:47:05] INFO: Saved 'requirements.csv' (72365 rows) to C:\Users\zalma\split
[2025-11-09 11:47:11] INFO: Saved 'platforms.csv' (112855 rows) to C:\Users\zalma\split
[2025-11-09 11:47:11] INFO: Saved 'game_platform.csv' (112855 rows) to C:\Users\zalma\split
[2025-11-09 11:

In [3]:
import pandas as pd

langs = pd.read_csv(r"C:\Users\zalma\split\languages.csv")
subs = pd.read_csv(r"C:\Users\zalma\split\game_subtitles.csv")

print("LANGS columns:", langs.columns.tolist())
print("SUBS columns:", subs.columns.tolist())


LANGS columns: ['id', 'name']
SUBS columns: ['appid', 'languageid']


In [13]:
import pandas as pd

langs = pd.read_csv(r"C:\Users\zalma\split\languages.csv")
subs = pd.read_csv(r"C:\Users\zalma\split\game_audio_language.csv")

appid = 292030

query = (
    subs.merge(langs, left_on="languageid", right_on="id", how="left")
         .query("appid == @appid")[["appid", "name"]]
)

print(query)

       appid                     name
3725  292030  Spanish - Latin America
3726  292030                   Polish
3727  292030       Simplified Chinese
3728  292030                   Korean
3729  292030      Portuguese - Brazil
3730  292030                  Russian
3731  292030                  English
3732  292030                   French
3733  292030                 Japanese
3734  292030                   German


In [ ]:
import pandas as pd

merged = pd.read_csv(r"C:\Users\zalma\merge\merged_master.csv", dtype=str)
game = pd.read_csv(r"C:\Users\zalma\split\game.csv", dtype=str)

missing = merged[~merged["appid"].isin(game["appid"])]
print(f"Hiányzó sorok száma: {len(missing)}")
display(missing[["appid", "name", "release_date"]])


In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/media.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/screenshots.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/movies.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/support.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/requirements.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/game_platform.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/platforms.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/game_package.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/packages.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/sub_package.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/game_developer.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/developers.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/game_publisher.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/publishers.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/game_genre.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/genres.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/game_category.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/categories.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/game_tag.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

csv_path = 'C:/Users/zalma/split/tags.csv'

try:
    df = pd.read_csv(csv_path)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")